### Quantify image-based metrics of TB stain with single-cell segmentation geometries

In [ ]:
import rasterio as rio
from rasterio.mask import mask
import geopandas as gpd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad
import cv2
import scipy
import scipy.stats as stats
from scipy.stats import normaltest
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from shapely.geometry import shape, box
from shapely.affinity import translate
from shapely.geometry import Polygon
from tqdm import tqdm
import scanpy as sc
import math
import scipy.io as sio
import gc


In [ ]:
def concatenate_h5ad_folder(input_folder,
                            output_folder='None',
                            name = 'concatenated_10x_output',
                            regex = ['donor','tissue','anatomical_position','method','sample','replicate','notes'],
                            regex_sep = '_',
                            counts_threshold=1000,
                            genes_threshold=100,
                            adata_file = 'None',
                            save_frequency = 100):
    
    """
    Take output files from decontx processing and concatenate into anndata object
    """
    
    if output_folder=='None':
        output_folder=input_folder
    
    if not os.path.isdir(input_folder):
        raise SyntaxError('Input folder cannot be found')   
        
    if not os.path.isdir(output_folder):
        raise SyntaxError('output folder cannot be found')   
    
    adata = sc.AnnData()
        
    files = [i for i in os.listdir(input_folder) if i.endswith(('.h5ad','.h5ad.gz'))]
    print("first file: ",files[0])
    count = 0
    for file in tqdm(files):
        
        
        info = file.split(".")[0] #assuming information is present in the filename
        
        #add something to check whether a given sample is already present in the adata file
        if adata_file!='None':
            if info in list(adata.obs['10X_run']):
                print('The sample ', info,' is already in the object. Skipping.')
                continue
        info_split = info.split(regex_sep)
        if len(info_split)<len(regex):
            print('Warning: the sample ',info,' does not have enough metadata information, ',regex[len(info_split):],' will be excluded')
            regex_fix = regex[:len(info_split)]
        else:
            regex_fix = regex
            
            
        print('reading ',file)
        adata_interim = sc.read_h5ad(os.path.join(input_folder,file))
        
        #add in .obs - this depends on the actual metadata format
        print('adding .obs columns to anndata object')
        
        adata_interim.obs['full_sample'] = info
           
        for n in range(0,len(regex_fix)):
            adata_interim.obs[regex_fix[n]] = info.split(regex_sep)[n]
        
        adata_interim.obs['cell_id'] = adata_interim.obs['full_sample'] + "_" + adata_interim.obs.index.astype('str')
        adata_interim.obs.set_index('cell_id',inplace=True)
        
        try:
            del adata_interim.obsm
        except:
            adata_interim
        
        for obs_column in adata.obs.columns:
            if "." in obs_column:
                adata.obs[obs_column] = adata.obs[obs_column].astype('str')
            
        if adata.shape[0]>1:  
            adata = ad.concat([adata,adata_interim],join='outer',merge='unique')
            print('added to adata')
        else:
            adata = adata_interim.copy()
            print('copying interim')
            
        adata.obs = adata.obs.astype('str')
        count = count+1
        
        if count == save_frequency:
            adata.write_h5ad(os.path.join(output_folder,name+'.h5ad'))
            count=0
        
        gc.collect()
            
    
    adata.write_h5ad(os.path.join(output_folder,name+'.h5ad'))
    
    return adata

In [ ]:
# Helper functions
def load_image_roi(file_name, window_coords):
    try:
        with rio.open(file_name) as aws_file:
            roi = aws_file.read(1, window=rio.windows.Window(*window_coords))
    except OSError:
        print(f"File read error on {file_name}")
        roi = np.zeros([window_coords[3],window_coords[2]])
        
    return np.squeeze(roi)

def convert_eight_bit(img):
    # Normalize contrast before reducing bit depth
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    norm_img = clahe.apply(img)    
    bg_subtract = norm_img - norm_img.min()
    
    if bg_subtract.max() == 0:
        return np.zeros(img.shape, dtype=np.uint8)
    else:
        range_norm = bg_subtract / bg_subtract.max()      
        return np.array(range_norm * 255, dtype=np.uint8)

In [ ]:
def load_tb_image_information(dataPath, experimentName, region):
    
    #initialize a dataframe
    df = pd.DataFrame()

    #iterate through the z planes - in this case there are 7
    for z in range(0,7):
        z_level = str(z)
        print('z_level: ', z_level)
    
        # Segmentation boundaries
        geometries = gpd.read_parquet(
            os.path.join(dataPath, experimentName, 'region_' + region, 'cell_boundaries.parquet')).set_index('EntityID')
    #print('Segmentation geometries loaded.')

    # TB stain channel image, AKA cellbound3
        TBmosaicPath = os.path.join(
            dataPath, experimentName, 'region_' + region, 'images', 'mosaic_Cellbound3_z' + z_level + '.tif')

    # DAPI stain channel image
        DAPImosaicPath = os.path.join(
            dataPath, experimentName, 'region_' + region, 'images', 'mosaic_DAPI_z' + z_level + '.tif')
    
    # Cellbound2 stain channel image
        Cellbound2mosaicPath = os.path.join(
            dataPath, experimentName, 'region_' + region, 'images', 'mosaic_Cellbound2_z' + z_level + '.tif')

    #print('Mosaic filepaths loaded.')

    # Micron to mosaic pixel transform
        transformationMatrix = np.loadtxt(
            os.path.join(dataPath, experimentName, 'region_' + region, 'images', 'micron_to_mosaic_pixel_transform.csv'))
    
    # Convert transformation matrix to correct affine transformation format for Geopandas
        t = transformationMatrix
        affine_matrix = [t[0,0], t[0,1], t[1,0], t[1,1], t[0,2], t[1,2]]

    #print('Micron to mosaic pixel transform loaded.')

    # Create new geodataframe column with mosaic pixel units
        geometries_mosaic = geometries.copy()
        geometries_mosaic['Geometry'] = geometries['Geometry'].affine_transform(affine_matrix)

    #print('Mosaic geometries calculated.')

    # Check that we have the right coordinates by overlaying with DAPI channel
    # What are the dimensions of the overall experiment?
        s = rio.open(DAPImosaicPath)
        print('Dimensions of experiment are ' + str(s.width) + ' x ' + str(s.height) + ' in pixels.')

        gdf = geometries_mosaic.copy()
        gdf = gdf[gdf['ZIndex'] == int(z_level)]

        shapes = list(gdf['Geometry'])
        masks = []

        with rio.open(TBmosaicPath) as src:
            for shape in tqdm(shapes):
                out_image, _ = mask(src, shape.geoms, crop=True)
                masks.append(out_image)
        
    # Save array of pixel intensities for each cell
        gdf['TB_intensities'] = pd.Series(dtype='object')
    #for i in tqdm(range(len(gdf))):
        #gdf.loc[:,'TB_intensities'].loc[gdf.index[i]] = masks[i][np.nonzero(masks[i])]
    
    
    #all nonzero TB intensities for each cell
        gdf['TB_intensities'] = [masks[i][np.nonzero(masks[i])] for i in range(len(gdf))] 
    
    #all TB intensities for each cell, including zeros
        gdf['TB_spatial'] = [masks[i].squeeze() for i in range(len(gdf))] 

    # Calculate some metrics

    # Sum signal
        gdf['sum_signal'] = [gdf.iloc[i]['TB_intensities'].sum() for i in range(len(gdf))]

    # Mean (excluding zeros)
        gdf['mean_signal'] = [gdf.iloc[i]['TB_intensities'].mean() for i in range(len(gdf))]
    
    # Median (excluding zeros)
        gdf['median_signal'] = [np.median(gdf.iloc[i]['TB_intensities']) for i in range(len(gdf))]
    
    # Standard deviation (excluding zeros)
        gdf['std_signal'] = [gdf.iloc[i]['TB_intensities'].std() for i in range(len(gdf))]

    # Average of top 100 values
        gdf['top_avg_signal'] = [np.sort(gdf.iloc[i]['TB_spatial'])[::-1][:100].mean() for i in range(len(gdf))]

    #create a data frame we can add to for each z plane:
        add = gdf[['ZIndex','sum_signal','mean_signal','median_signal','std_signal','top_avg_signal']]
        df = pd.concat([df,add])
    return df
    
def make_average_gdf(df,adata):

#find the mean values for each entity across z positions
#add a binary classifier to adata
    avg_signal_df = pd.DataFrame(columns=df.columns)
    for ind in tqdm(list(set(df.index))):
        sub = df.loc[ind]
        avg = sub.mean(axis=0)
        avg_signal_df.loc[ind] = avg

    avg_signal_df.index = [str(i) for i in avg_signal_df.index]

    #subset to only cells that are included in the final object
    avg_signal_df['Entity_ID'] = avg_signal_df.index.astype('str').copy()
    avg_signal_df = avg_signal_df[avg_signal_df.Entity_ID.isin(list(adata.obs.index))].copy()
    avg_signal_df.set_index('Entity_ID',inplace=True)
    print(avg_signal_df.shape)

    return avg_signal_df


In [ ]:
def load_vizgen_h5ad(processedDataPath,experimentName, filename, region,
                    min_volume=200, max_volume=5000, min_counts=20, min_dapi = 2000000, TB_threshold = 700000):
    
    #read in the h5ad file
    adata = sc.read_h5ad(processedDataPath+experimentName+'/'+filename)

    #print the number of cells, cells per region, and show the object
    print('number of cells in unfiltered dataset: ',adata.shape[0])
    print('cells per region: ', adata.obs.region.value_counts())

    #filter to one region at a time
    adata = adata[adata.obs.region==region].copy()
    adata

    # Calculate total counts
    adata.obs['total_counts'] = adata.X.toarray().sum(axis=1)
    adata.obs['log_total_counts'] = np.log(adata.obs['total_counts'] + 1)
    
    #print graphs
    #Create thresholds based on the plots above, and plot those lines onto the data. 
    fig, axes = plt.subplots(1,4, figsize=(12,4))

    # Cell volume
    bins = np.logspace(0,4,20)
    ax = axes[0]
    ax.hist(adata.obs['volume'], bins=bins,cumulative=True,density=True)
    ax.set_xlabel('cell volume (um^3)')
    ax.set_ylabel('count')
    ax.set_xscale('log')
    ax.axvline(min_volume,color='red')
    ax.axvline(max_volume,color='black')
    ax.axhline(0.05,color='red')

    # Total transcripts per cell
    bins = np.logspace(0,3.5,20)
    ax = axes[1]
    ax.hist(adata.obs['total_counts'], bins=bins,cumulative=True,density=True)
    ax.set_xlabel('total transcripts')
    ax.set_ylabel('count')
    ax.set_xscale('log')
    ax.axvline(min_counts,color='red')
    ax.axhline(0.20,color='red')


    # DAPI signal per cell
    bins = np.logspace(2,8,20)
    ax = axes[2]
    ax.hist(adata.obs['DAPI_high_pass'], bins=bins,cumulative=True,density=True)
    ax.set_xlabel('DAPI')
    ax.set_ylabel('fraction of cells')
    ax.set_xscale('log')
    #ax.set_yscale('log')
    ax.axvline(min_dapi,color='red')
    ax.axhline(0.1,color='red')

    # TB signal per cell
    bins = np.logspace(2,6,20)
    ax = axes[3]
    ax.hist(adata.obs['Cellbound3_high_pass'], bins=bins,cumulative=True,density=True)
    ax.set_xlabel('TB stain')
    ax.set_ylabel('count')
    ax.set_xscale('log')
    ax.axhline(0.95,color='black')
    ax.axvline(TB_threshold,color='black')

    print('saving figure')
    fig.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/'+experimentName+'region'+str(region)+'_basicQC.pdf')
    print("size of adata object: ", adata.shape)
    return adata

def filter_h5ad(adata,min_volume=200, max_volume=5000, min_counts=20, min_dapi = 2000000):
    #filter the object based on the thresholds above
    cells = adata.shape[0]
    adata = adata[(adata.obs['volume'] > min_volume) &
         (adata.obs['volume'] < max_volume) &
         (adata.obs['total_counts'] > min_counts) &
         (adata.obs['DAPI_high_pass'] > min_dapi)].copy()
    print('Filtering complete: ' + str(len(adata)) + ' cells remaining out of ' + str(cells) + ' original cells.')
    return adata

def show_spatial_plots(adata, clipx, experimentName, region,figsize = (10,10)):
    #Show the section and add lines for any removal of damage
    fig, ax = plt.subplots(figsize=figsize)
    sc.pl.spatial(adata, color='Cellbound3_high_pass', spot_size=50, ax=ax,color_map='OrRd',vmax=5000000)

    fig, ax = plt.subplots(figsize=figsize)
    ax.axvline(clipx,color='red')
    ax.axvline(clipx+1000,color='blue')

    if experimentName == '202309251210_20230925-RitwicqA-TB_VMSC02401' and region=='1':
        ax.axhline(6500,color='red')
        ax.axhline(7500,color='blue')
    sc.pl.spatial(adata, color='log_total_counts', spot_size=30, ax=ax)
    
def sc_preprocess(adata):
    # Basic scanpy preprocessing
    adata.X = adata.layers['counts'].copy()
    sc.pp.normalize_total(adata,target_sum=100)
    sc.pp.log1p(adata)

    #save normalized data
    adata.layers['log_normalized'] = adata.X.copy()
    print('log normalized')

    #scale data
    sc.pp.scale(adata)
    adata.layers['scale_data'] = adata.X.copy()
    print('scaled')

    #run dimensional reduction
    print('calculating pcs')
    sc.pp.pca(adata)

    print('calculating umap')
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.tl.leiden(adata, resolution=0.5)

    print('Scanpy processing complete.')
    #check that counts remain
    print(adata.layers['counts'][:1,:5])
    return adata

def tbcells_only_calculate_distance_to_tb(adata):
    adata.obs['distance_to_tb'] = -100
    #calculate euclidean distance from each of these cells to every other cell, find the minimum
    tb_cells = adata.obs.index[adata.obs.TB_class_from_images=='High']
    
    #iterate over all tb cells
    #find the location of the cell in space
    #X_spatial is not an x-coordinate, it's a point
    
    for cell in tqdm(list(tb_cells)):
        dist = []
        cell_loc = np.round(list([adata.obs.center_x[cell],adata.obs.center_y[cell]]),decimals=2)
    
        #iterate over all TB+ cells and find the distance from each one to the cell in question
        for tb_cell in tb_cells:
            tb_cell_loc = np.round(list([adata.obs.center_x[tb_cell],adata.obs.center_y[tb_cell]]),decimals=2)
            new_dist = np.round(math.dist(cell_loc,tb_cell_loc),decimals=2)
            dist.append(new_dist)

        #find the minimum distance to the 2nd tb+ cell
        try:
            min_dist = sorted(dist,reverse=False)[2]
        except:
            min_dist = 401
            print('not enough cells to calculate min distance')
        
        #add this to the object
        adata.obs.loc[cell,'distance_to_tb'] = min_dist
        
    #show any outliers in terms of cellbound 3 (tb) or distance to tb, print a graph
    print("cells with TB signal above 5 million:", adata[adata.obs.Cellbound3_high_pass>5e6].shape)
    sc.pl.scatter(adata[adata.obs.TB_class_from_images=='High'],x='center_x',y='distance_to_tb',color='Cellbound3_high_pass')
    
    return adata

#calculate euclidean distance from each of the tb+ cells to every other cell, find the minimum
def calculate_distance_to_tb(adata):
    if 'High' in set(adata.obs.TB_class_from_images):

        adata.obs['distance_to_tb'] = -100


        tb_cells = adata.obs.index[adata.obs.TB_class_from_images=='High']
        count=1
    #iterate over all cells
        for cell in tqdm(list(adata.obs.index)):
            dist = []
    
        #if the cell is TB+, distance = 0
            if adata.obs.loc[cell,'TB_class_from_images']=='High':
                adata.obs.loc[cell,'distance_to_tb'] = 0
                continue
        
            cell_loc = np.round(list([adata.obs.center_x[cell],adata.obs.center_y[cell]]),decimals=2)
    #if the cell is not TB+, calculate its distance from each TB+ cell
    #iterate over all TB+ cells and find the distance from each one to the cell in question
            for tb_cell in tb_cells:
                tb_cell_loc = np.round(list([adata.obs.center_x[tb_cell],adata.obs.center_y[tb_cell]]),decimals=2)
                new_dist = np.round(math.dist(cell_loc,tb_cell_loc),decimals=2)
                dist.append(new_dist)

    #find the minimum distance
            min_dist = min(dist)
    
    #add this to the object
            adata.obs.loc[cell,'distance_to_tb'] = min_dist
    else:
        adata.obs['distance_to_tb'] = 5000
    return adata

#run a permutation test comparing each transcript from within the binsize relative to a tb+ cell against all cells
def permutation_test_for_tb_loc(adata,layer='log_normalized',binsize=100,perm_iterations=1000):

    if isinstance(adata.layers[layer], scipy.sparse._csr.csr_matrix):
        adata_df = pd.DataFrame.sparse.from_spmatrix(adata.layers[layer])
        adata_df = adata_df.sparse.to_dense()
    else:
        adata_df = pd.DataFrame(adata.layers[layer])
    adata_df.columns = adata.var.index
    adata_df.index = adata.obs.index
    adata_df['distance_real'] = adata.obs['distance_to_tb'].copy()
    adata_df.columns = [i.replace('-','') for i in adata_df.columns]

    adata_df['Bin'] = 'Null'

    #for the first 4 bins, plot the distribution on top of this one in a different color
    adata_df.loc[(adata_df.distance_real>=6*binsize) & (adata_df.distance_real<7*binsize),'Bin'] = 'Bin7'
    adata_df.loc[(adata_df.distance_real>=3*binsize) & (adata_df.distance_real<4*binsize),'Bin'] = 'Bin4'
    adata_df.loc[(adata_df.distance_real>=2*binsize) & (adata_df.distance_real<3*binsize),'Bin'] = 'Bin3'
    adata_df.loc[(adata_df.distance_real>=binsize) & (adata_df.distance_real<2*binsize),'Bin'] = 'Bin2'
    adata_df.loc[adata_df.distance_real<binsize,'Bin'] = 'Bin1'
    adata_df.loc[adata_df.distance_real==0,'Bin'] = 'Bin0'
    print("number of cells per bin: ",adata_df.Bin.value_counts())
    adata_df = adata_df.sort_values('Bin',ascending=True)

    #permutation test ~43 iterations/second
    df_results = pd.DataFrame(columns=['sample_stat','average_stat','p_value'])
    for transcript in tqdm(adata_df.columns[:140]):
        ref_average = np.mean(adata_df.loc[~adata_df['Bin'].isin(['Bin1','Bin0']),transcript])
        #print(ref_average)
                              
        sample_stat = np.mean(adata_df.loc[adata_df['Bin']=='Bin1',transcript]) - ref_average #calculate the difference in means between your bin1 distribution and the null distribution
        stats = np.zeros(perm_iterations)
        for k in range(perm_iterations):
            labels = np.random.permutation((adata_df['Bin'] == 'Bin1').values) #randomly rearrange the bin labels so you get another sample in 'Bin1'
            stats[k] = np.mean(adata_df[transcript][labels]) - ref_average #for each of 1000 iterations, calculate the difference between the random Bin1 Nos2 values and the null distribution
    
        if sample_stat>=0:
            p_value = np.mean(stats > sample_stat)
        if sample_stat<0:
            p_value = np.mean(stats < sample_stat)
    #save the stat and p-value per transcript
        df_results.loc[transcript] = [sample_stat,np.mean(stats),p_value] 

    df_results = df_results.sort_values('p_value',ascending=True)
    return adata_df, df_results

In [ ]:
def plot_tb_altered_transcripts(sc_data,pval_threshold, stat_threshold, df_results,experimentName, region,perm_iterations):
    top_hits = df_results[(df_results.p_value<pval_threshold)].sort_values('sample_stat',ascending=False)
    top_hits = list(top_hits[top_hits.sample_stat>stat_threshold].index)
    top_hits = [i for i in top_hits if not i.startswith('Blank')]

    bottom_hits =df_results[(df_results.p_value<pval_threshold) & (df_results.sample_stat< -stat_threshold)].index
    bottom_hits = [i for i in bottom_hits if not i.startswith('Blank')]

    #show violin or feature plots of these genes in the smartseq data
   
    if len(top_hits)>2:
        sc.pl.heatmap(sc_data,top_hits,groupby='leiden',show_gene_labels=True)
        #sc.pl.stacked_violin(sc_data,top_hits,groupby='leiden',rotation=90,title = 'Upregulated near TB')
    if len(bottom_hits)>2:
        sc.pl.heatmap(sc_data,bottom_hits,groupby='leiden',show_gene_labels=True)
        #sc.pl.stacked_violin(sc_data,bottom_hits,groupby='leiden',rotation=90,title='Downregulated near TB')
    
    allprobes = [i for i in list(df_results.index) if not i.startswith('Blank')]

    sc_data.var['UpNearTB'] = False
    sc_data.var.loc[sc_data.var.index.isin(top_hits),'UpNearTB'] = True

    sc_data.var['DownNearTB'] = False
    sc_data.var.loc[sc_data.var.index.isin(bottom_hits),'DownNearTB'] = True

    sc_data.var['UnchangedNearTB'] = False
    sc_data.var.loc[sc_data.var.index.isin(allprobes),'UnchangedNearTB'] = True
    sc_data.var.loc[sc_data.var['UpNearTB']==True,'UnchangedNearTB'] = False
    sc_data.var.loc[sc_data.var['DownNearTB']==True,'UnchangedNearTB'] = False

    print("up near TB: ", len(top_hits), "\ndown near TB:", len(bottom_hits))

    sc.pp.calculate_qc_metrics(sc_data,qc_vars=['UpNearTB','DownNearTB','UnchangedNearTB'],inplace=True)

    fig,ax = plt.subplots(figsize=(5,3))
    ax.set_ylim(0,7)
    #sc.pl.violin(sc_data,['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'],groupby='leiden',rotation=90)
    sc.pl.violin(sc_data,['pct_counts_UpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = experimentName+'region'+str(region)+'ViolinPlot_UP_TB.pdf',ax=ax)
    fig,ax = plt.subplots(figsize=(5,3))
    ax.set_ylim(0,7)
    sc.pl.violin(sc_data,['pct_counts_DownNearTB'],groupby='leiden',rotation=90,inner='quartile',save = experimentName+'region'+str(region)+experimentName+'region'+str(region)+'ViolinPlot_Down_TB.pdf',ax=ax)

    fig,ax = plt.subplots(figsize=(5,3))
    ax.set_ylim(0,7)
    sc.pl.violin(sc_data,['pct_counts_UnchangedNearTB'],groupby='leiden',rotation=90,inner='quartile',save = experimentName+'region'+str(region)+'ViolinPlot_NOCHANGE_TB.pdf',ax=ax)

    print(df_results.p_value[df_results.p_value>0].min())
    df_results.loc[df_results.p_value==0,'p_value'] = 0.9/perm_iterations
    df_results.to_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats'+experimentName+'_region'+str(region)+'.csv')

    plt.figure(figsize=(20,10))
    plt.scatter(x = df_results.sample_stat, y=-np.log10(df_results.p_value))
    plt.xlabel('difference in means, Bin1 - Null')
    plt.ylabel('-log p value')

    for i in range(df_results.shape[0]):
        plt.text(x=df_results.sample_stat[i],y=-np.log10(df_results.p_value[i]),s=df_results.index[i], 
              fontdict=dict(color='black',size=10))
    
    plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/volcano'+experimentName+'_region'+str(region)+'.pdf')
    return sc_data

def plot_cumulative_dist(marker_dict,adata_df,experimentName,region,ymax = 0):
#run for each set of marker genes
    for celltype in list(marker_dict.keys()):
   
        for transcript in marker_dict[celltype]:
            if ymax==0:
                ymax_new = max(adata_df[transcript])
            else:
                ymax_new=ymax
    #if transcript not in ['Nos2','C1qa','C1qb','C1qc','Fcgr4','Lyve1']:
        #continue
  
            fig,ax = plt.subplots(figsize=(5,3))
            sns.histplot(data = adata_df,
                 x=transcript, 
                 bins=10000, 
                 binrange=(0,ymax_new),
                 stat="density",
                 element="step", 
                 fill=False, 
                 cumulative=True,
                color='Grey',
                alpha=0.5)
        
    
            sns.histplot(data = adata_df[adata_df.Bin!='Null'],
                 x=transcript, 
                 hue='Bin', 
                 bins=10000, 
                 binrange=(0,ymax_new),
                 stat="density",
                 element="step", 
                 fill=False, 
                 cumulative=True, 
                 common_norm=False,
                 palette = ['Black','Red','Orange','Green','Teal','Blue'])
            ax.set_xlabel(celltype)
            ax.set_ylabel('Cumulative Distribution')
            ax.set_title(transcript)
            ax.set_ylim(0,1)
            fig.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/'+experimentName+'_region'+str(region)+'_'+transcript+'_histogram.pdf') 

In [ ]:
def assign_tb_pos_cells(adata,avg_signal_df, threshold, std_threshold,experimentName, region):
    #add metrics to adata object
    adata.obs.loc[avg_signal_df.index,'TB_std'] = avg_signal_df.std_signal
    adata.obs.loc[avg_signal_df.index,'TB_mean'] = avg_signal_df.mean_signal
    adata.obs.loc[avg_signal_df.index,'TB_topaverage'] = avg_signal_df.top_avg_signal

    if experimentName=='202309251210_20230925-RitwicqA-TB_VMSC02401' and region=='2':
        threshold = 250
    
    adata.obs['TB_class_from_images'] = 'High'
    adata.obs.loc[adata.obs.TB_topaverage<=threshold,'TB_class_from_images'] = 'Low'
    print('initial thresholding',adata.obs.TB_class_from_images.value_counts(sort=False))
    adata.obs.loc[(adata.obs.TB_std<=std_threshold)&(adata.obs.TB_topaverage>threshold),'TB_class_from_images'] = 'Removed'
    print('with sd cutoff',adata.obs.TB_class_from_images.value_counts(sort=False))

    #run distance calculation
    adata = tbcells_only_calculate_distance_to_tb(adata)

    #print out initial counts, remove cells with high TB values and lone cells
    adata.obs.loc[adata.obs.Cellbound3_high_pass>5e6,'TB_class_from_images']='Removed'
    print('removed super high fluorescent cells', adata.obs.TB_class_from_images.value_counts(sort=False))
    adata.obs.loc[adata.obs.distance_to_tb>400,'TB_class_from_images']='Removed' 
    print("removed lone cells", adata.obs.TB_class_from_images.value_counts(sort=False))

    if experimentName=='202305261458_TB-TEST3-MB_VMSC02401':
        adata.obs.loc[adata.obs.center_x>4000,'TB_class_from_images'] = 'Removed'
        print(adata.obs.TB_class_from_images.value_counts())
    
    adata.obs.TB_class_from_images.to_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Cell_annotation_csvs/'+experimentName+'region'+str(region)+'TB_binary.csv')
    adata.obs.TB_class_from_images.value_counts()   
    
    fig, ax = plt.subplots(figsize=(20,10))
    sc.pl.spatial(adata,color='TB_class_from_images',spot_size=30,palette=['red','lightgrey','grey'],ax=ax)

    sc.pl.violin(adata,['center_x','center_y'],groupby='TB_class_from_images')
    sc.pl.violin(adata,['TB_std','TB_mean'],groupby='TB_class_from_images')
    sc.pl.violin(adata,['TB_topaverage','Cellbound3_high_pass'],groupby='TB_class_from_images')
    sc.pl.scatter(adata,x='TB_topaverage',y='TB_std',color='TB_class_from_images',size=30)

    fig,ax = plt.subplots(figsize=(10,10))
    ax.axvline(8000,color='black')
    ax.axvline(9000,color='black') #1 mm
    sc.pl.spatial(adata,spot_size=20,color = 'TB_class_from_images',palette=['red','lightgrey','grey'],ax=ax)
    fig.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/'+experimentName+'region'+str(region)+'1mmscale_TBclass.pdf')       
    return adata

****1. Load in Shoshana's final smartseq object, remove neutrophils, and edit gene names:****

In [ ]:
sc_data = sc.read_h5ad('/hpc/projects/data_lg/shoshana/Smartseq2nd/adataMNPNEW.h5ad')
sc_data = sc_data[sc_data.obs.leiden!='Neutrophil'].copy()
sc.pl.umap(sc_data,color = 'leiden')
sc_data.var_names = [i.replace('-','') for i in sc_data.var_names]

****2. Filter and QC the Vizgen h5ad file****

In [ ]:
#make a table
experiment_df = pd.DataFrame(columns = ['experimentName','region','clipx'])
experiment_df.loc[0] = ['202305231432_TB-CTRL1-MB_VMSC02401','0',0]
experiment_df.loc[1] = ['202305231432_TB-CTRL1-MB_VMSC02401','1',0]
experiment_df.loc[2] = ['202305231432_TB-CTRL1-MB_VMSC02401','2',0]
experiment_df.loc[3] = ['202305251147_TB-CTRL2-MB_VMSC02401','0',0]
experiment_df.loc[4] = ['202305251147_TB-CTRL2-MB_VMSC02401','1',0]
experiment_df.loc[5] = ['202305261458_TB-TEST3-MB_VMSC02401','1',2000]
experiment_df.loc[7] = ['202309251210_20230925-RitwicqA-TB_VMSC02401','1',7500]
experiment_df.loc[8] = ['202309251210_20230925-RitwicqA-TB_VMSC02401','2',2500]
experiment_df.loc[9] = ['202310091253_AmandaS-TB-02_VMSC02401','0',0]
experiment_df.loc[10] = ['202310091253_AmandaS-TB-02_VMSC02401','1',0]
experiment_df.loc[11] = ['202310091253_AmandaS-TB-02_VMSC02401','2',0]
experiment_df.loc[12] = ['202310111428_RitwicqA-TB-4b_VMSC02401','1',0]
experiment_df.loc[13] = ['202310111428_RitwicqA-TB-4b_VMSC02401','2',0]
experiment_df

Set all variables

In [ ]:
dataPath = '/hpc/projects/genomics/MERSCOPE/preprocessed_data'
processedDataPath = '/hpc/projects/genomics/MERSCOPE/postprocessed_data/ProcessedCounts/'

threshold = 300
std_threshold = 150 

layer = 'normalized_to_polyT'
binsize=100
perm_iterations=10000
pval_threshold = 0.001
stat_threshold = 0.15

marker_dict = {'High':['Nos2', 'H2Ab1', 'Irf1','C1qb'], 'Low':['Sparc','Jun','Lyve1'],'Neither':['Irf9','H2K1','Cxcr2']}

## Run all the code on each section

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for jloc in experiment_df.index:
    experimentName = experiment_df.experimentName[jloc]
    region = experiment_df.region[jloc]
    clipx = experiment_df.clipx[jloc]
    print(experimentName, region, clipx)

    filenames = os.listdir(processedDataPath+experimentName)
    filename = [i for i in filenames if i.endswith('.h5ad')][0]
    
    ## Process the vizgen adata file and filter out damaged tissue
    adata = load_vizgen_h5ad(processedDataPath = processedDataPath,experimentName=experimentName, filename = filename, region=region,
                    min_volume=200, max_volume=5000, min_counts=20, min_dapi = 2000000, TB_threshold = 700000)
    adata = filter_h5ad(adata)
    show_spatial_plots(adata,clipx, experimentName,region)

    #remove any damage
    adata = adata[adata.obs.center_x>clipx].copy()
    if experimentName == '202309251210_20230925-RitwicqA-TB_VMSC02401' and region=='1':
        adata = adata[adata.obs.center_y<6500].copy()
    sc.pl.spatial(adata,spot_size=50)
    adata = sc_preprocess(adata)
    
    ## Load in the image data and calculate metrics for each segment using the antibody information
    
    #collect data from images
    df = load_tb_image_information(dataPath=dataPath,experimentName=experimentName,region=region)
    #calculate average signal per cell for each metric
    avg_signal_df = make_average_gdf(df,adata)
    print(avg_signal_df.shape, adata.shape)

    #Define TB positive cells
    adata = assign_tb_pos_cells(adata,avg_signal_df, threshold,std_threshold,experimentName, region)

    #normalize count data to polyA signal per cell
    adata.layers['normalized_to_polyT'] = adata.layers['counts']/np.array(adata.obs.PolyT_high_pass).reshape(-1,1)*np.mean(adata.obs.PolyT_high_pass)
    adata.layers['normalized_to_polyT'] = sparse.csr_matrix(adata.layers['normalized_to_polyT'])
        
    #skip these parts if there are not TB+ cells
    if 'High' in set(adata.obs.TB_class_from_images):
        
        adata = calculate_distance_to_tb(adata)
        adata.obs.distance_to_tb = adata.obs.distance_to_tb.astype('float')

        adata_df, df_results = permutation_test_for_tb_loc(adata,layer=layer,binsize=binsize,perm_iterations=perm_iterations)
        sc_data = plot_tb_altered_transcripts(sc_data,pval_threshold, stat_threshold, df_results,experimentName, region,perm_iterations)

        plot_cumulative_dist(marker_dict,adata_df,experimentName,region,ymax=50)
    adata.write_h5ad('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Annotated_h5ad/'+experimentName+'_region'+str(region)+'_Annotated.h5ad')
    gc.collect()

In [ ]:
#show Dnase1l3 expression 
sc.pl.violin(adata,'Dnase1l3',groupby='TB_class_from_images')

In [ ]:
adata_tb = adata[adata.obs.Condition=='TB'].copy()
print('Dnase1l3')
for group in list(set(adata_tb.obs.grouped_distance_to_tb)):
    far = adata_tb[adata_tb.obs.grouped_distance_to_tb==group,'Dnase1l3']
    #calculate percent with positive Dnase1l3 counts
    original = far.shape[0]
    pos = far[far.layers['normalized_to_polyT']>1].shape[0]
    print(group,pos/original*100)
 

print('Nos2')
for group in list(set(adata_tb.obs.grouped_distance_to_tb)):
    far = adata_tb[adata_tb.obs.grouped_distance_to_tb==group,'Nos2']
    #calculate percent with positive Dnase1l3 counts
    original = far.shape[0]
    pos = far[far.layers['normalized_to_polyT']>1].shape[0]
    print(group,pos/original*100)
    

In [ ]:
sc.pl.violin(sc_data,['Dnase1l3','Nos2'],groupby='leiden',rotation=90)

In [ ]:
#load in final df_results and find consensus genes, plot top ones with a dot for each replicate, show consensus on violin plots for smartseq data
#for cumulative distribution plots, show on total dataset

#1. make a file with all the genes and all the data points


folder = '/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/new_individual/tb/'
df = pd.DataFrame(columns = ['gene','stat','p_value','experiment','region'])

for file in os.listdir(folder):
    if not file.endswith('.csv'):
        continue
    experiment = file.split('_')[1]
    region = file.split('_')[3].split('.')[0]
    
    stats = pd.read_csv(folder+file)
    stats.columns = ['gene','stat','remove','p_value']
    del stats['remove']
    stats['experiment'] = experiment
    stats['region'] = region

    df = pd.concat([df, stats],ignore_index=True)

#plot each gene using a swarmplot
average_df = pd.DataFrame(columns=['average'])
for gene in list(set(df.gene)):
    mean = np.mean(df.loc[df.gene==gene].stat)
    average_df.loc[gene] = mean
    
average_df = average_df.sort_values('average')
gene_list = list(average_df.index)

df['average_value'] = 0
for gene in average_df.index:
    df.loc[df.gene==gene,'average_value']=average_df.loc[gene,'average']
df
df = df.sort_values('average_value')
df.p_value[df.p_value>0.0001] = 0.1
df.p_value[df.p_value<0.0001] = 0.0001
df.p_value.value_counts()

df.to_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/statsAllTB_individualReplicates-excludingTB.csv')
ax = sns.catplot(data=df.iloc[700:1125], 
                 x="stat", 
                 y="gene", 
                 hue="experiment", 
                 #palette = ['red','orange','lightgrey'],
                 kind="swarm",
                 aspect=1,
                 height=10,
                 row_order=gene_list)
plt.savefig("/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/new/replicate_samplestat_experiment_last700samples-excludingtb.pdf", format='pdf')
plt.show()



ax = sns.catplot(data=df, 
                 x="stat", 
                 y="gene", 
                 hue="experiment", 
                 #palette = ['red','orange','lightgrey'],
                 kind="swarm",
                 aspect=1,
                 height=10,
                 row_order=gene_list)

plt.savefig("/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/new/replicate_samplestat_experiment-withoutTBcells.pdf", format='pdf')
plt.show()

In [ ]:
gene_list

In [ ]:
df_new = df.copy()
df_new.loc[df_new.gene.str.startswith("Blank"),'gene']='Blank'
df_new = df_new.loc[~df_new.gene.isin(neither)]
df_new

ax = sns.catplot(data=df_new, 
                 x="stat", 
                 y="gene", 
                 hue="experiment", 
                 #palette = ['red','orange','lightgrey'],
                 kind="swarm",
                 aspect=1,
                 height=10,
                 row_order=gene_list)

plt.savefig("/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/new/replicate_samplestat_experiment-withoutTBcells.pdf", format='pdf')
plt.show()

In [ ]:
average_df

#from kruskal-wallis test (for upregulated genes) p<0.01 compared to all the blanks

#clean up these markers
low_markers = list(average_df.loc[average_df.average< -0.2].index)
print(low_markers)
high_markers = ['Ifng','Socs1','Cd40','C1qc','Atf3','Ifi30','Adam19','Cd2','Tnf','Cxcr4','C1qb','Junb','Socs3',
                'Nfkb2','Csf3r','Il1b','Irf7','Ifngr1','Sparc','Clec7a','Irf5','Jak2','Tgfb1','H2D1','Lgals3',
                'Coro1b','Spn','Clec4e','Lamp1','Sirpa','H2Eb1','Hmox1','Hif1a','Itgal','Coro1a','Stat1','Irf1','Ly6e','Nos2','H2Ab1',
                'H2K1','Cd74']


neither = [x for x in list(average_df.index) if x not in low_markers+high_markers]
neither = [x for x in neither if not x.startswith('Blank')]

heatmap_labels = {'Up':high_markers,'Down':low_markers}
heatmap_labels

In [ ]:
#print a heatmap
sc.pl.heatmap(sc_data,var_names = heatmap_labels, groupby='leiden',show_gene_labels=True,save = 'tbfinal_cleanmarkers.pdf')

sc_data.var.UpNearTB=False
sc_data.var.UnchangedNearTB=False
sc_data.var.DownNearTB=False

sc_data.var.loc[high_markers,'UpNearTB']=True
sc_data.var.loc[low_markers,'DownNearTB']=True
sc_data.var.loc[neither,'UnchangedNearTB']=True

sc.pp.calculate_qc_metrics(sc_data,qc_vars = ['UpNearTB','DownNearTB','UnchangedNearTB'],percent_top=None,inplace=True,log1p=False)
sc.pl.violin(sc_data,['pct_counts_UpNearTB','pct_counts_DownNearTB','pct_counts_UnchangedNearTB'],groupby='leiden',rotation=90,inner='quartile',save ='rip.pdf')

In [ ]:
sc.pl.violin(sc_data,['pct_counts_UpNearTB','pct_counts_DownNearTB','pct_counts_UnchangedNearTB'],groupby='leiden',rotation=90,inner='quartile', save ='rip.pdf')



In [ ]:
sc_data.layers

In [ ]:
#load all the h5ads and merge them into one object, re-normalize

folder = '/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/'

adata = concatenate_h5ad_folder(input_folder=folder+'Annotated_h5ad',
                            output_folder=folder,
                            name = 'concatenated_vizgen_h5ad',
                            regex = ['date','slide','project','region'],
                            regex_sep = '_',
                            counts_threshold=0,
                            genes_threshold=0,
                            adata_file = 'None',
                            save_frequency = 100)

print(adata.obs['slide'].value_counts())

adata.obs['Condition'] = 'Control'
adata.obs.loc[~adata.obs.slide.isin(['TB-CTRL2-MB','TB-CTRL1-MB']),'Condition']='TB'
print(adata.obs.groupby(['Condition','slide'])['fov'].count())

adata.obs['genomics_personnel'] = 'Michael_Borja'
adata.obs.loc[adata.obs.slide.str.startswith('Amanda'),'genomics_personnel'] = 'Amanda_Seng'
adata.obs.loc[adata.obs.slide.str.startswith('Ritwicq'),'genomics_personnel'] = 'Ritwicq_Arjyal'
adata.obs.loc[adata.obs.slide.str.startswith('20230925'),'genomics_personnel'] = 'Ritwicq_Arjyal'

adata.obs['experiment_name'] = 'Control_1'
adata.obs.loc[adata.obs.slide.str.startswith('20230925'),'experiment_name'] = 'TB_Mouse2_SectionA'
adata.obs.loc[adata.obs.slide.str.startswith('Ritwicq'),'experiment_name'] = 'TB_Mouse4_SectionB'
adata.obs.loc[adata.obs.slide.str.startswith('Amanda'),'experiment_name'] = 'TB_Mouse4_SectionC'
adata.obs.loc[adata.obs.slide.str.startswith('TB-TEST3'),'experiment_name'] = 'TB_MouseX_Test3'
adata.obs.loc[adata.obs.slide.str.startswith('TB-CTRL2'),'experiment_name'] = 'Ctrl_MouseX_Exp2'
adata.obs.loc[adata.obs.slide.str.startswith('TB-CTRL1'),'experiment_name'] = 'Ctrl_MouseX_Exp1'

print(adata.obs.experiment_name.value_counts())

adata.obs['full_sample'] = adata.obs.experiment_name.astype('str')+'_'+adata.obs.region.astype('str')
adata.obs.full_sample.value_counts()

adata.obs.PolyT_high_pass = adata.obs.PolyT_high_pass.astype('float')
adata.layers['normalized_to_polyT'] = adata.layers['counts']/np.array(adata.obs.PolyT_high_pass).reshape(-1,1)*np.mean(adata.obs.PolyT_high_pass)
adata.layers['normalized_to_polyT'] = sparse.csr_matrix(adata.layers['normalized_to_polyT'])
       
    
adata.obs.distance_to_tb = adata.obs.distance_to_tb.astype('float')
adata.obs['grouped_distance_to_tb'] = '1_far'
adata.obs.loc[adata.obs.distance_to_tb<1000,'grouped_distance_to_tb'] = '2_within_1_mm'
adata.obs.loc[adata.obs.distance_to_tb<100,'grouped_distance_to_tb'] = '3_within_100_um'
adata.obs.loc[adata.obs.distance_to_tb==0,'grouped_distance_to_tb'] = '4_TBPositive'
adata.obs.loc[adata.obs.distance_to_tb<-1,'grouped_distance_to_tb'] = '5_Unclear'

adata.obs.grouped_distance_to_tb.value_counts()
adata.var.index = [i.replace("-","") for i in adata.var.index]

#save this object with all the columns

adata.write_h5ad(folder+'Total_annotated_vizgen_TB_20231120.h5ad')

adata_keep = adata.copy()



Run code on the full adata object (TB only)

In [ ]:
experimentName='AllTB'
region='All'
perm_iterations=3000
binsize=100

pval_threshold = 0.001
stat_threshold = 0.5

marker_dict = {'up':['Cd74','Sparc'], #['Cd74','H2K1','Nos2','C1qb','Hmox1']
              'down':['Lyve1','Dusp1'],
              'neither':['Cx3cr1','Ms4a7']}

layer = 'normalized_to_polyT'

adata_tb = adata[adata.obs.Condition=='TB'].copy()

#adata_df, df_results = permutation_test_for_tb_loc(adata_tb,layer,binsize,perm_iterations)
#sc_data = plot_tb_altered_transcripts(sc_data,pval_threshold, stat_threshold, df_results,experimentName, region,perm_iterations)
plot_cumulative_dist(marker_dict,adata_df,experimentName,region,ymax=50)

In [ ]:
adata.obs.groupby(['experiment_name','Condition'])['fov'].count()

In [ ]:
marker_dict = {'up':['Cd74','H2K1','Nos2','C1qb','Hmox1'],
              'down':['Lyve1','Dusp1'],
              'neither':['Cx3cr1','Ms4a7','Apoe']}

layer = 'normalized_to_polyT'

#adata_tb = adata[adata.obs.Condition=='TB'].copy()

#adata_df, df_results = permutation_test_for_tb_loc(adata_tb,layer,binsize,perm_iterations)
#sc_data = plot_tb_altered_transcripts(sc_data,pval_threshold, stat_threshold, df_results,experimentName, region,perm_iterations)
plot_cumulative_dist(marker_dict,adata_df,experimentName,region,ymax=20)

In [ ]:
sc_data.var.index[sc_data.var.UpNearTB]

In [ ]:
sc_data.var.index[sc_data.var.DownNearTB]

In [ ]:
region = 'all_ymax_10'



sc_data = plot_tb_altered_transcripts(sc_data,pval_threshold, stat_threshold, df_results,experimentName, region,perm_iterations)
plot_cumulative_dist(marker_dict,adata_df,experimentName,region,ymax=50)

In [ ]:
experimentName

In [ ]:
region = 'all_ymax_10'
marker_dict = {'up':['Cd74','H2K1','Nos2','C1qb','Hmox1'],
              'down':['Lyve1','Dusp1'],
              'neither':['Fabp4','Krt79']}
plot_cumulative_dist(marker_dict,adata_df,experimentName,region,ymax=100)

In [ ]:
#from calculations with all samples together
#note: top 30 genes from this method and the previous method are completely overlapping except for c1qb


up_merged = sc_data.var.index[sc_data.var.UpNearTB]
up_merged

In [ ]:
#from calculations with each sample separate 
#top 40

up_separate = average_df.sort_values('average',ascending=False)[:30].index
up_separate

In [ ]:
for gene in list(up_merged):
    if gene in list(up_separate):
        print(gene, "is in both lists")
    else:
        print(gene, "is unique to the new calculation excluding TB+ cells")

In [ ]:
sc_data.var.index[sc_data.var.DownNearTB]

In [ ]:
#run test once more (maybe with differential expression? on normalized to polyT data, comparing the within 200 um to within 1 mm and far categories)
adata.X = adata.layers['normalized_to_polyT'].copy()
sc.pp.log1p(adata)
adata_test = adata[adata.obs.Condition=='TB'].copy()
adata_test.obs.grouped_distance_to_tb = adata_test.obs.grouped_distance_to_tb.astype('object')
sc.tl.rank_genes_groups(adata_test,
                        groupby='grouped_distance_to_tb',
                        groups=['3_within_100_um','4_TBPositive','1_far'],
                        reference = '2_within_1_mm')

df_ranked = sc.get.rank_genes_groups_df(adata_test, 
                                group='3_within_100_um',
                                key='rank_genes_groups', 
                                pval_cutoff=None, 
                                log2fc_min=None, 
                                log2fc_max=None, 
                                gene_symbols=None)



In [ ]:
sc.pl.rank_genes_groups(adata_test)

In [ ]:
#take the top 10 genes and plot them across the adata categories

sc.pl.stacked_violin(adata[adata.obs.Condition=='TB'],
                     gene_list[100:140],
                     groupby='grouped_distance_to_tb',
                     layers='normalized_to_polyT')



In [ ]:
#plot counts of the highest de genes
stats = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats_Total_annotated_vizgen_TB_20231102_counts.csv',index_col=0)
stats = stats.sort_values('sample_stat',ascending=False)
stats[:30] #test statistic >1

#try with all the genes
genelist = list(stats.index)


#grab cells that are within 100 um and in the TB object
adata_keep.obs['distance_to_tb'] = adata_keep.obs['distance_to_tb'].astype('float')
keep_cells = adata_keep[adata_keep.obs.Condition=='TB'].copy()
keep_cells.var.index = [i.replace('-','') for i in keep_cells.var.index]

#set up a dataframe to collect correlation coefficients
correlation_frame = pd.DataFrame(index=genelist,columns=genelist)
from scipy.stats import pearsonr

for i in range(0,len(genelist)):
    gene_a = genelist[i]
    gene_a_values = keep_cells[:,gene_a].layers['counts'].todense()
    x = pd.DataFrame(gene_a_values)[0]
    for j in range(0,len(genelist)):
        gene_b = genelist[j]
        gene_b_values = keep_cells[:,gene_b].layers['counts'].todense()
        
        y = pd.DataFrame(gene_b_values)[0]
        pearson_coef, _ = np.round(pearsonr(x,y),decimals = 2)
        correlation_frame.loc[gene_a,gene_b] = pearson_coef
        if pearson_coef > 0.6:
            print("Pearson correlation coefficient between",gene_a,"and", gene_b, ":", pearson_coef)
            #sc.pl.scatter(keep_cells,x=gene_a, y = gene_b, color = 'distance_to_tb',layers='counts')

    
print(correlation_frame)

#make a heatmap
data = correlation_frame.astype('float')
plt.imshow( data )
plt.title( "Correlation matrix between top 30 upregulated genes using raw counts" )

 
# Set tick labels
plt.xticks(range(len(data.columns)),\
           data.columns, rotation=90)
plt.yticks(range(len(data.columns)),
           data.columns)

plt.show()




In [ ]:
#make a heatmap
data = correlation_frame.astype('float')
plt.figure(figsize=(30,30))
plt.imshow( data )
plt.title( "Correlation matrix between top 30 upregulated genes using raw counts" )

 
# Set tick labels
plt.xticks(range(len(data.columns)),\
           data.columns, rotation=90)
plt.yticks(range(len(data.columns)),
           data.columns)

plt.show()


In [ ]:
#which genes are correlated? 

In [ ]:
#plot one gene at a time in the violin plots
sc_data = sc.read_h5ad('/hpc/projects/data_lg/shoshana/Smartseq2nd/adataMNPNEW.h5ad')
sc_data = sc_data[sc_data.obs.leiden!='Neutrophil'].copy()
sc.pl.umap(sc_data,color = 'leiden')
sc_data.var_names = [i.replace('-','') for i in sc_data.var_names]

sc.pl.stacked_violin(sc_data,genelist[:30],groupby='leiden',swap_axes=True)
sc.pl.stacked_violin(sc_data,genelist[123:140],groupby='leiden',swap_axes=True)

In [ ]:
#plot counts of the lowest de genes
stats = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats_Total_annotated_vizgen_TB_20231102_counts.csv',index_col=0)
stats = stats.sort_values('sample_stat',ascending=True)
print(stats[:10]) #test statistic <= -0.3

genelist = list(stats.index[:30])
print(genelist)    

for i in range(0,len(genelist)):
    gene_a = genelist[i]
    for j in range(i+1,len(genelist)):
        gene_b = genelist[j]
        sc.pl.scatter(keep_cells,x=gene_a, y = gene_b, color = 'distance_to_tb',layers='counts')
    
   

In [ ]:
sc.pl.scatter(keep_cells,x='Adam19',y='H2-K1')

In [ ]:
keep_cells.var

In [ ]:
#repeating with raw counts to see if I get the same results
#could also repeat by comparing the 100 um to some other distance

#run the permutation test on A) all the controls, B) all the TB cells
condition='TB'
adata = adata_keep[adata_keep.obs.Condition==condition].copy()

adata_df = pd.DataFrame.sparse.from_spmatrix(adata.layers['counts'])
adata_df = adata_df.sparse.to_dense()
adata_df.columns = adata.var.index
adata_df.index = adata.obs.index
adata_df['distance_real'] = adata.obs['distance_to_tb'].copy()
adata_df.columns = [i.replace('-','') for i in adata_df.columns]

adata_df['Bin'] = 'Null'
binsize = 100

#for the first 4 bins, plot the distribution on top of this one in a different color
adata_df.distance_real = adata_df.distance_real.astype('float')
adata_df.loc[(adata_df.distance_real>=6*binsize) & (adata_df.distance_real<7*binsize),'Bin'] = 'Bin7'
adata_df.loc[(adata_df.distance_real>=3*binsize) & (adata_df.distance_real<4*binsize),'Bin'] = 'Bin4'
adata_df.loc[(adata_df.distance_real>=2*binsize) & (adata_df.distance_real<3*binsize),'Bin'] = 'Bin3'
adata_df.loc[(adata_df.distance_real>=binsize) & (adata_df.distance_real<2*binsize),'Bin'] = 'Bin2'
adata_df.loc[adata_df.distance_real<binsize,'Bin'] = 'Bin1'
print("number of cells per bin: ",adata_df.Bin.value_counts())
adata_df = adata_df.sort_values('Bin',ascending=True)

#permutation test ~43 iterations/second
perm_iterations = 1000
df_results = pd.DataFrame(columns=['sample_stat','average_stat','p_value'])
for transcript in tqdm(adata_df.columns[:140]):
    sample_stat = np.mean(adata_df.loc[adata_df['Bin']=='Bin1',transcript]) - np.mean(adata_df[transcript]) #calculate the difference in means between your bin1 distribution and the null distribution
    stats = np.zeros(perm_iterations)
    for k in range(perm_iterations):
        labels = np.random.permutation((adata_df['Bin'] == 'Bin1').values) #randomly rearrange the bin labels so you get another sample in 'Bin1'
        stats[k] = np.mean(adata_df[transcript][labels]) - np.mean(adata_df[transcript]) #for each of 1000 iterations, calculate the difference between the random Bin1 Nos2 values and the null distribution

    if sample_stat>=0:
        p_value = np.mean(stats > sample_stat)
    if sample_stat<0:
        p_value = np.mean(stats < sample_stat)
    #save the stat and p-value per transcript
    df_results.loc[transcript] = [sample_stat,np.mean(stats),p_value] 

df_results = df_results.sort_values('p_value',ascending=True)

#increased close to TB
top_hits = df_results[(df_results.p_value<0.001)].sort_values('sample_stat',ascending=False)
top_hits = list(top_hits[top_hits.sample_stat>0].index)
top_hits = [i for i in top_hits if not i.startswith('Blank')]

bottom_hits =df_results[(df_results.p_value<0.001) & (df_results.sample_stat<0)].index
bottom_hits = [i for i in bottom_hits if not i.startswith('Blank')]

#show violin or feature plots of these genes in the smartseq data
sc_data = sc.read_h5ad('/hpc/projects/data_lg/shoshana/Smartseq2nd/adataMNPNEW.h5ad')
sc_data = sc_data[sc_data.obs.leiden!='Neutrophil'].copy()
sc.pl.umap(sc_data,color = 'leiden')
sc_data.var_names = [i.replace('-','') for i in sc_data.var_names]

sc.pl.stacked_violin(sc_data,top_hits,groupby='leiden',rotation=90)

allprobes = [i for i in list(df_results.index) if not i.startswith('Blank')]

sc_data.var['UpNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(top_hits),'UpNearTB'] = True

sc_data.var['DownNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(bottom_hits),'DownNearTB'] = True

sc_data.var['NotUpNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(allprobes),'NotUpNearTB'] = True
sc_data.var.loc[sc_data.var['UpNearTB']==True,'NotUpNearTB'] = False
sc_data.var.loc[sc_data.var['DownNearTB']==True,'NotUpNearTB'] = False

sc.pl.heatmap(sc_data,top_hits,groupby='leiden')
sc.pp.calculate_qc_metrics(sc_data,qc_vars=['UpNearTB','DownNearTB','NotUpNearTB'],inplace=True)
sc.pl.umap(sc_data,color = ['leiden'])
sc.pl.umap(sc_data,color = ['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'])
sc.pl.umap(sc_data,color = ['total_counts_NotUpNearTB','pct_counts_NotUpNearTB'])

fig,ax = plt.subplots(figsize=(5,3))
ax.set_ylim(0,7)
#sc.pl.violin(sc_data,['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'],groupby='leiden',rotation=90)
sc.pl.violin(sc_data,['pct_counts_UpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_UP_TB.pdf',ax=ax)
fig,ax = plt.subplots(figsize=(5,3))
ax.set_ylim(0,7)
sc.pl.violin(sc_data,['pct_counts_DownNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_Down_TB.pdf',ax=ax)

fig,ax = plt.subplots(figsize=(5,3))
ax.set_ylim(0,7)
sc.pl.violin(sc_data,['pct_counts_NotUpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_NOTUP_TB.pdf',ax=ax)
             
print(df_results.p_value[df_results.p_value>0].min())
df_results.loc[df_results.p_value==0,'p_value'] = 0.9/perm_iterations


df_results.to_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats_'+condition+'rawcounts.csv')

plt.scatter(x = df_results.sample_stat, y=-np.log10(df_results.p_value))
plt.xlabel('difference in means, Bin1 - Null')
plt.ylabel('-log p value')

for i in range(df_results.shape[0]):
 plt.text(x=df_results.sample_stat[i],y=-np.log10(df_results.p_value[i]),s=df_results.index[i], 
          fontdict=dict(color='black',size=4))

plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/volcano_'+condition+'rawcounts.pdf')

print(df_results.sort_values('sample_stat',ascending=True)[:20])
print(df_results.sort_values('sample_stat',ascending=False)[:20])   


marker_dict = {'High':['Nos2', 'H2Ab1', 'Irf1','C1qb'], 'Low':['Sparc','Jun','Lyve1'],'Neither':['Irf9','H2K1','Cxcr2']}


#run for each set of marker genes
for celltype in list(marker_dict.keys()):
   
    for transcript in marker_dict[celltype]:
    #if transcript not in ['Nos2','C1qa','C1qb','C1qc','Fcgr4','Lyve1']:
        #continue

        
  
        fig,ax = plt.subplots(figsize=(5,3))
        sns.histplot(data = adata_df,
                 x=transcript, 
                 bins=10000, 
                 binrange=(0,max(adata_df[transcript])),
                 stat="density",
                 element="step", 
                 fill=False, 
                 cumulative=True,
                color='Grey',
                alpha=0.5)
        
    
        sns.histplot(data = adata_df[adata_df.Bin!='Null'],
                 x=transcript, 
                 hue='Bin', 
                 bins=10000, 
                 binrange=(0,max(adata_df[transcript])),
                 stat="density",
                 element="step", 
                 fill=False, 
                 cumulative=True, 
                 common_norm=False,
                palette = ['Red','Orange','Green','Teal','Blue'])
        ax.set_xlabel(celltype)
        ax.set_ylabel('Cumulative Distribution')
        ax.set_title(transcript)
        ax.set_ylim(0,1)
        fig.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/'+condition+'_'+transcript+'rawcounts_histogram.pdf')        
        
print("Up: ",sc_data.var.UpNearTB.value_counts())
print("Down: ", sc_data.var.DownNearTB.value_counts())
print("Neither: ",sc_data.var.NotUpNearTB.value_counts())

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
sc.pl.umap(sc_data,color='leiden',ax=ax,size=100,save='umap.pdf')

In [ ]:
#directly compare the two 
adata = adata_keep.copy()
adata

In [ ]:
set(adata.obs.full_sample)

In [ ]:
df = pd.DataFrame(adata.obs.groupby(['full_sample','TB_class_from_images'])['slide'].count())
newdf = pd.DataFrame(index = list(set(adata.obs.full_sample)),columns = ['experiment','High','Low','Removed'])
for i,j in df.index:
    exp = i 
    tb_class = j
    value = df.loc[i,j][0]
    newdf.loc[i,j] = value
    
newdf.experiment = newdf.index.copy()
newdf

In [ ]:
newdf_small = newdf[['experiment','High']]
newdf_small = newdf_small.sort_values("High")
index_order = newdf_small.index

In [ ]:
#plot a bar plot
newdf_small.plot(x='experiment', kind='bar', stacked=True,
        title='Number of cells classified as TB+ by section',color = 'red')
plt.tight_layout()
plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/TB_pos_cells_by_sample_bar.pdf',format='pdf')
plt.show()

In [ ]:
newdf['total'] = newdf.High+newdf.Low+newdf.Removed
newdf_small = newdf[['experiment','total']]
newdf_small = newdf_small.loc[index_order]
newdf_small

In [ ]:
#plot a bar plot
newdf_small.plot(x='experiment', kind='bar', stacked=True,
        title='Total number of cells by section',color = 'grey')
plt.tight_layout()
plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/all_cells_by_sample_bar.pdf',format='pdf')

plt.show()

In [ ]:
newdf_small = newdf[['experiment','High','total']]
newdf_small['percent'] = newdf_small['High']*100/newdf_small['total']
newdf_small = newdf_small[['experiment','percent']]
newdf_small = newdf_small.loc[index_order]
newdf_small

In [ ]:
#plot a bar plot
newdf_small.plot(x='experiment', kind='bar', stacked=True,
        title='Total number of cells by section',color = 'grey')
plt.tight_layout()
plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/percent_tb_cells_by_sample_bar.pdf',format='pdf')

plt.show()

In [ ]:
#load in both data frames and compare the gene signatures
ctrl_stats = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats_Control.csv',index_col=0)
ctrl_stats

In [ ]:
#load in both data frames and compare the gene signatures
tb_stats = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats_TB.csv',index_col=0)
tb_stats

In [ ]:
#for every gene, plot the tb sample_stat vs ctrl sample_stat
#make a combined data frame
all_stats = pd.DataFrame(index = tb_stats.index)
all_stats['tb_sample_stat'] = tb_stats['sample_stat'].copy()
all_stats['tb_pval'] = tb_stats['p_value'].copy()

for i in all_stats.index:
    all_stats.loc[i,'Ctrl_sample_stat'] = ctrl_stats.loc[i,'sample_stat']
    all_stats.loc[i,'Ctrl_sample_pval'] = ctrl_stats.loc[i,'p_value']
    
all_stats

In [ ]:
#sort by tb_sample_stat and plot both tb and control stats
all_stats = all_stats.sort_values('tb_sample_stat')
all_stats['gene'] = all_stats.index.copy()

#plot a volcano plot
fig, ax = plt.subplots(figsize=(30,10))
ax.scatter(x = all_stats.tb_sample_stat, y=-np.log10(all_stats.tb_pval))
ax.set_xlabel('difference in means, Bin1 - Null')
ax.set_ylabel('-log p value')
ax.set_title('TB volcano plot')

for i in range(all_stats.shape[0]):
 ax.text(x=all_stats.tb_sample_stat[i],y=-np.log10(all_stats.tb_pval[i]),s=all_stats.gene[i], 
          color='black',size=8,rotation=30)

print(all_stats.loc[abs(all_stats.tb_sample_stat)>0.1])

In [ ]:
#plot a volcano plot
fig, ax = plt.subplots(figsize=(30,10))
ax.scatter(x = all_stats.Ctrl_sample_stat, y=-np.log10(all_stats.Ctrl_sample_pval))
ax.set_xlabel('difference in means, Bin1 - Null')
ax.set_ylabel('-log p value')
ax.set_title('Ctrl volcano plot')

for i in range(all_stats.shape[0]):
 ax.text(x=all_stats.Ctrl_sample_stat[i],y=-np.log10(all_stats.Ctrl_sample_pval[i]),s=all_stats.gene[i], 
          color='black',size=8,rotation=30)

In [ ]:
#make a table comparing the genes that are significantly altered in the TB samples vs those significantly altered in the control samples
tb_up = all_stats.gene[all_stats.tb_sample_stat>0.05]
tb_down = all_stats.gene[all_stats.tb_sample_stat<-0.05]
tb_neither = all_stats.gene[(all_stats.tb_sample_stat<0.05)&(all_stats.tb_sample_stat>-0.05)]

In [ ]:
print(len(tb_up))
x = all_stats.loc[tb_up,'Ctrl_sample_stat']>0.05
all_stats.loc[x[x].index]

In [ ]:
print(len(tb_down))
x = all_stats.loc[tb_down,'Ctrl_sample_stat']<-0.05
all_stats.loc[x[x].index]

In [ ]:
tb_up = tb_up[~tb_up.isin(['C1qa','C1qb'])]
tb_down = tb_down[~tb_down.isin(['Lyve1','Dusp1','Itga6','Scarf2','Ly6e','Atf4','Col4a1','H2D1','Car4','Lamp1'])]

In [ ]:
#re-run the violin plots with these genes
allprobes = [i for i in list(all_stats.index) if not i.startswith('Blank')]

sc_data.var['UpNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(tb_up),'UpNearTB'] = True

sc_data.var['DownNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(tb_down),'DownNearTB'] = True

sc_data.var['NotUpNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(allprobes),'NotUpNearTB'] = True
sc_data.var.loc[sc_data.var['UpNearTB']==True,'NotUpNearTB'] = False
sc_data.var.loc[sc_data.var['DownNearTB']==True,'NotUpNearTB'] = False

sc.pl.heatmap(sc_data,tb_up,groupby='leiden',show_gene_labels=True)
sc.pp.calculate_qc_metrics(sc_data,qc_vars=['UpNearTB','DownNearTB','NotUpNearTB'],inplace=True)
sc.pl.umap(sc_data,color = ['leiden'])
sc.pl.umap(sc_data,color = ['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'])
sc.pl.umap(sc_data,color = ['total_counts_NotUpNearTB','pct_counts_NotUpNearTB'])

fig,ax = plt.subplots(figsize=(5,3))
ax.set_ylim(0,7)
#sc.pl.violin(sc_data,['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'],groupby='leiden',rotation=90)
sc.pl.violin(sc_data,['pct_counts_UpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_UP_limit01_strict_TB.pdf',ax=ax)
fig,ax = plt.subplots(figsize=(5,3))
ax.set_ylim(0,7)
sc.pl.violin(sc_data,['pct_counts_DownNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_Down_limit01_strict_TB.pdf',ax=ax)

fig,ax = plt.subplots(figsize=(5,3))
ax.set_ylim(0,7)
sc.pl.violin(sc_data,['pct_counts_NotUpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_NEITHER_limit01_strict_TB.pdf',ax=ax)
             

In [ ]:
ctrl_stats

In [ ]:
#which genes are consistently up- or down-regulated across all of the TB experiments? 
i=0
for file in os.listdir('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/individual/tb/'):
    #print(file.split("_",maxsplit=1)[1].split(".")[0])
    stats = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/individual/tb/'+file,index_col=0)
    upregulated = stats.loc[(stats.sample_stat>0.02)&(stats.p_value<0.01)].index
    if i==0:
        shared_upregulated=upregulated.copy()
    else:
        shared_upregulated = shared_upregulated[shared_upregulated.isin(upregulated)]
shared_upregulated

In [ ]:
#which genes are consistently up- or down-regulated across all of the TB experiments? 
i=0
for file in os.listdir('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/individual/tb/'):
    #print(file.split("_",maxsplit=1)[1].split(".")[0])
    stats = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/individual/tb/'+file,index_col=0)
    downregulated = stats.loc[(stats.sample_stat<-0.02)&(stats.p_value<0.01)].index
    if i==0:
        shared_downregulated=downregulated.copy()
    else:
        shared_downregulated = shared_downregulated[shared_downregulated.isin(downregulated)]
shared_downregulated

In [ ]:
sc_data.obs.leiden.value_counts()

In [ ]:
#run violin plots with these genes
#re-run the violin plots with these genes
allprobes = [i for i in list(all_stats.index) if not i.startswith('Blank')]

sc_data.var['UpNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(shared_upregulated),'UpNearTB'] = True

sc_data.var['DownNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(shared_downregulated),'DownNearTB'] = True

sc_data.var['NotUpNearTB'] = False
sc_data.var.loc[sc_data.var.index.isin(allprobes),'NotUpNearTB'] = True
sc_data.var.loc[sc_data.var['UpNearTB']==True,'NotUpNearTB'] = False
sc_data.var.loc[sc_data.var['DownNearTB']==True,'NotUpNearTB'] = False

sc.pp.calculate_qc_metrics(sc_data,qc_vars=['UpNearTB','DownNearTB','NotUpNearTB'],inplace=True)

fig,ax = plt.subplots(figsize=(2,3))
ax.set_ylim(0,6)
#sc.pl.violin(sc_data,['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'],groupby='leiden',rotation=90)
sc.pl.violin(sc_data[sc_data.obs.leiden.isin(['moDerived Macrophage','Immature_moDerived Macrophage'])],['pct_counts_UpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_UP_shared_TB.pdf',ax=ax)
fig,ax = plt.subplots(figsize=(2,3))
ax.set_ylim(0,6)
sc.pl.violin(sc_data[sc_data.obs.leiden.isin(['moDerived Macrophage','Immature_moDerived Macrophage'])],['pct_counts_DownNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_Down_shared_TB.pdf',ax=ax)

fig,ax = plt.subplots(figsize=(2,3))
ax.set_ylim(0,6)
sc.pl.violin(sc_data[sc_data.obs.leiden.isin(['moDerived Macrophage','Immature_moDerived Macrophage'])],['pct_counts_NotUpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_NEITHER_shared_TB.pdf',ax=ax)
             

In [ ]:
x = sc.pl.violin(sc_data[sc_data.obs.leiden.isin(['moDerived Macrophage','Immature_moDerived Macrophage'])],['pct_counts_NotUpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = 'AllFiles_'+condition+'_ViolinPlot_NEITHER_shared_TB.pdf')

In [ ]:
adata.obs.distance_to_tb = adata.obs.distance_to_tb.astype('float')

In [ ]:
#grab the TB-infected cells and directly compare them to their neighbors using pseudobulk
adata = adata_keep[adata_keep.obs.Condition=='TB'].copy()
adata.obs.distance_to_tb = adata.obs.distance_to_tb.astype('float')
tb_cells = adata.obs.index[adata.obs.TB_class_from_images=='High']
neighbors = adata.obs.index[(adata.obs.distance_to_tb<15)&(adata.obs.distance_to_tb>0)]

print(len(tb_cells),len(neighbors))

In [ ]:
#can grab the points directly from sc_data
enriched_percentages = sc_data.obs.loc[sc_data.obs.leiden.isin(['moDerived Macrophage','Immature_moDerived Macrophage'])]
enriched_percentages[['pct_counts_UpNearTB','pct_counts_DownNearTB','pct_counts_NotUpNearTB','leiden']]

#run a t-test for each one comparing the leiden clusters to each other

In [ ]:
#grap the 95 percentiles from each one
#up

TBup_immature = enriched_percentages.pct_counts_UpNearTB.loc[enriched_percentages.leiden=='Immature_moDerived Macrophage']
median = TBup_immature.median()
upper_bound_95 = np.quantile(TBup_immature,.75)
lower_bound_95 = np.quantile(TBup_immature,.25)
print("Percentage of TB-induced transcripts in immature mo-derived Macrophages: 25th percentile, median, 75th percentile: ", lower_bound_95, median, upper_bound_95)

TBup_mature = enriched_percentages.pct_counts_UpNearTB.loc[enriched_percentages.leiden=='moDerived Macrophage']
median = TBup_mature.median()
upper_bound_95 = np.quantile(TBup_mature,.75)
lower_bound_95 = np.quantile(TBup_mature,.25)
print("Percentage of TB-induced transcripts in mature mo-derived Macrophages: 25th percentile, median, 75th percentile: ", lower_bound_95, median, upper_bound_95)

#down

TBdown_immature = enriched_percentages.pct_counts_DownNearTB.loc[enriched_percentages.leiden=='Immature_moDerived Macrophage']
median = TBdown_immature.median()
upper_bound_95 = np.quantile(TBdown_immature,.75)
lower_bound_95 = np.quantile(TBdown_immature,.25)
print("Percentage of TB-suppressed transcripts in immature mo-derived Macrophages: 25th percentile, median, 75th percentile: ", lower_bound_95, median, upper_bound_95)

TBdown_mature = enriched_percentages.pct_counts_DownNearTB.loc[enriched_percentages.leiden=='moDerived Macrophage']
median = TBdown_mature.median()
upper_bound_95 = np.quantile(TBdown_mature,.75)
lower_bound_95 = np.quantile(TBdown_mature,.25)
print("Percentage of TB-suppressed transcripts in mature mo-derived Macrophages: 25th percentile, median, 75th percentile: ", lower_bound_95, median, upper_bound_95)


#neither

TBneither_immature = enriched_percentages.pct_counts_NotUpNearTB.loc[enriched_percentages.leiden=='Immature_moDerived Macrophage']
median = TBneither_immature.median()
upper_bound_95 = np.quantile(TBneither_immature,.75)
lower_bound_95 = np.quantile(TBneither_immature,.25)
print("Percentage of TB-agnostic transcripts in immature mo-derived Macrophages: 25th percentile, median, 75th percentile: ", lower_bound_95, median, upper_bound_95)

TBneither_mature = enriched_percentages.pct_counts_NotUpNearTB.loc[enriched_percentages.leiden=='moDerived Macrophage']
median = TBneither_mature.median()
upper_bound_95 = np.quantile(TBneither_mature,.75)
lower_bound_95 = np.quantile(TBneither_mature,.25)
print("Percentage of TB-agnostic transcripts in mature mo-derived Macrophages: 25th percentile, median, 75th percentile: ", lower_bound_95, median, upper_bound_95)


Run this part with all the cells together to see how it looks
Might be worth running pseudobulk

In [ ]:
#pull out the correct cells
TB_cells = adata.obs.index[adata.obs.distance_to_tb==0]
Near_cells = adata.obs.index[(adata.obs.distance_to_tb>0)&(adata.obs.distance_to_tb<12)]
print(len(TB_cells),len(Near_cells))

keep_cells = list(TB_cells)+list(Near_cells)
adata_sub = adata[adata.obs.index.isin(keep_cells)].copy()
adata_sub.obs.loc[TB_cells,'distance_group'] = 'TB_infected'
adata_sub.obs.loc[Near_cells,'distance_group'] = 'uninfected'
adata_sub.obs.distance_group.value_counts()

In [ ]:
#run DE in mast
column = 'distance_group'
layer = 'log_normalized'
out_folder = '/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/exported_formast/'
#1. save files for opening in R
sio.mmwrite(out_folder +"sparse_matrix.mtx",adata_sub.layers[layer])
adata_sub.obs[column].astype('str').to_csv(out_folder+'obs.csv')
pd.DataFrame(adata_sub.var.index).to_csv(out_folder+'var.csv',index=0)

#conda activate popv
#Rscript /hpc/projects/data.science/leah.dorman/notebooks/Spatial/Vizgen/mast.R

In [ ]:
#from mast tutorial (without seurat)

In [ ]:
#load from mast
rankedgenes = pd.read_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/exported_formast/distance_group_TB_infected_vs_uninfected_MAST.csv')
rankedgenes


In [ ]:
#volcano
#change this plot by assigning zeros to a number smaller than the smallest p value possible
plt.scatter(x = rankedgenes.coef, y=-np.log10(rankedgenes.fdr))
plt.xlabel('log fold change, TB infected vs neighbors')
plt.ylabel('-log p value')
plt.xlim(-0.6,0.6)
plt.axvline(0, color='black')
#which labels to plot
threshold = 0.1
for i in range(rankedgenes.shape[0]):
    if abs(rankedgenes.coef[i])>threshold:
        plt.text(x=rankedgenes.coef[i],y=-np.log10(rankedgenes.fdr[i]),s=rankedgenes.primerid[i], fontdict=dict(color='black',size=8))

plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/TBinfected_vs_neighbors_volcano.pdf')

In [ ]:
#how do these genes compare to the single cell experiment? 
up_in_tb = rankedgenes.primerid.loc[(rankedgenes.fdr<0.01)&(rankedgenes.coef>0.1)]
down_in_tb = rankedgenes.primerid.loc[(rankedgenes.fdr<0.01)&(rankedgenes.coef<-0.1)]
down_in_tb

In [ ]:
sc_data.var['UpINTB'] = False
sc_data.var['DownINTB'] = False
sc_data.var.UpINTB.loc[sc_data.var.index.isin(up_in_tb)]=True
sc_data.var.DownINTB.loc[sc_data.var.index.isin(down_in_tb)]=True

#calculate qc metrics
sc.pp.calculate_qc_metrics(sc_data,qc_vars=['UpINTB','DownINTB'],inplace=True)

sc.pl.violin(sc_data,['pct_counts_UpINTB','pct_counts_DownINTB'],groupby='leiden',rotation=90)

In [ ]:
#I think for this analysis to work you actually have to pull out the macrophages

In [ ]:
# #do most transcripts scale with cell size? 
# #plot total counts vs transcript counts
# adata.X = adata.layers['counts'].copy()
# for transcript in adata.var.index[:20]:
#     sc.pl.scatter(adata,x='total_counts',y=transcript,color='experiment_name')



In [ ]:
#could try loading in each each experiment from the folder, running the permutation test on raw counts, finding the merged;
folder = '/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Annotated_h5ad/'
for file in os.listdir(folder):
    adata = sc.read_h5ad(folder+file)
    experiment = file.split(".")[0]
    print(experiment)
    #repeating with raw counts to see if I get the same results
#could also repeat by comparing the 100 um to some other distance

#run the permutation test on A) all the controls, B) all the TB cells
# condition='TB'
# adata = adata_keep[adata_keep.obs.Condition==condition].copy()
    layer = 'counts'

    adata_df = pd.DataFrame.sparse.from_spmatrix(adata.layers[layer])
    adata_df = adata_df.sparse.to_dense()
    adata_df.columns = adata.var.index
    adata_df.index = adata.obs.index
    adata_df['distance_real'] = adata.obs['distance_to_tb'].copy()
    adata_df.columns = [i.replace('-','') for i in adata_df.columns]

    adata_df['Bin'] = 'Null'
    binsize = 100

#for the first 4 bins, plot the distribution on top of this one in a different color
    adata_df.distance_real = adata_df.distance_real.astype('float')
    adata_df.loc[(adata_df.distance_real>=6*binsize) & (adata_df.distance_real<7*binsize),'Bin'] = 'Bin7'
    adata_df.loc[(adata_df.distance_real>=3*binsize) & (adata_df.distance_real<4*binsize),'Bin'] = 'Bin4'
    adata_df.loc[(adata_df.distance_real>=2*binsize) & (adata_df.distance_real<3*binsize),'Bin'] = 'Bin3'
    adata_df.loc[(adata_df.distance_real>=binsize) & (adata_df.distance_real<2*binsize),'Bin'] = 'Bin2'
    adata_df.loc[adata_df.distance_real<binsize,'Bin'] = 'Bin1'
    print("number of cells per bin: ",adata_df.Bin.value_counts())
    adata_df = adata_df.sort_values('Bin',ascending=True)

#permutation test ~43 iterations/second
    perm_iterations = 1000
    df_results = pd.DataFrame(columns=['sample_stat','average_stat','p_value'])
    for transcript in tqdm(adata_df.columns[:140]):
        sample_stat = np.mean(adata_df.loc[adata_df['Bin']=='Bin1',transcript]) - np.mean(adata_df[transcript]) #calculate the difference in means between your bin1 distribution and the null distribution
        stats = np.zeros(perm_iterations)
        for k in range(perm_iterations):
            labels = np.random.permutation((adata_df['Bin'] == 'Bin1').values) #randomly rearrange the bin labels so you get another sample in 'Bin1'
            stats[k] = np.mean(adata_df[transcript][labels]) - np.mean(adata_df[transcript]) #for each of 1000 iterations, calculate the difference between the random Bin1 Nos2 values and the null distribution
    
        if sample_stat>=0:
            p_value = np.mean(stats > sample_stat)
        if sample_stat<0:
            p_value = np.mean(stats < sample_stat)
    #save the stat and p-value per transcript
        df_results.loc[transcript] = [sample_stat,np.mean(stats),p_value] 

    df_results = df_results.sort_values('p_value',ascending=True)

#increased close to TB
    top_hits = df_results[(df_results.p_value<0.001)].sort_values('sample_stat',ascending=False)
    top_hits = list(top_hits[top_hits.sample_stat>0.1].index)
    top_hits = [i for i in top_hits if not i.startswith('Blank')]

    bottom_hits =df_results[(df_results.p_value<0.001) & (df_results.sample_stat<-0.1)].index
    bottom_hits = [i for i in bottom_hits if not i.startswith('Blank')]

#show violin or feature plots of these genes in the smartseq data
    sc_data = sc.read_h5ad('/hpc/projects/data_lg/shoshana/Smartseq2nd/adataMNPNEW.h5ad')
    sc_data = sc_data[sc_data.obs.leiden!='Neutrophil'].copy()
    #sc.pl.umap(sc_data,color = 'leiden')
    sc_data.var_names = [i.replace('-','') for i in sc_data.var_names]

    try:
        sc.pl.stacked_violin(sc_data,top_hits,groupby='leiden',rotation=90)
    except:
        print('too little data for stacked violin')
    allprobes = [i for i in list(df_results.index) if not i.startswith('Blank')]

    sc_data.var['UpNearTB'] = False
    sc_data.var.loc[sc_data.var.index.isin(top_hits),'UpNearTB'] = True

    sc_data.var['DownNearTB'] = False
    sc_data.var.loc[sc_data.var.index.isin(bottom_hits),'DownNearTB'] = True

    sc_data.var['NotUpNearTB'] = False
    sc_data.var.loc[sc_data.var.index.isin(allprobes),'NotUpNearTB'] = True
    sc_data.var.loc[sc_data.var['UpNearTB']==True,'NotUpNearTB'] = False
    sc_data.var.loc[sc_data.var['DownNearTB']==True,'NotUpNearTB'] = False

    try:
        sc.pl.heatmap(sc_data,top_hits,groupby='leiden')
    except:
        print('too little data')
    sc.pp.calculate_qc_metrics(sc_data,qc_vars=['UpNearTB','DownNearTB','NotUpNearTB'],inplace=True)
    #sc.pl.umap(sc_data,color = ['leiden'])
    #sc.pl.umap(sc_data,color = ['total_counts_UpNearTB','pct_counts_UpNearTB','total_counts_DownNearTB','pct_counts_DownNearTB'])
    #sc.pl.umap(sc_data,color = ['total_counts_NotUpNearTB','pct_counts_NotUpNearTB'])

    fig,ax = plt.subplots(figsize=(5,3))
    ax.set_ylim(0,7)
    sc.pl.violin(sc_data,['pct_counts_UpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = experiment+'_'+layer+'_ViolinPlot_UP_TB.pdf',ax=ax)
    fig,ax = plt.subplots(figsize=(5,3))
    ax.set_ylim(0,7)
    sc.pl.violin(sc_data,['pct_counts_DownNearTB'],groupby='leiden',rotation=90,inner='quartile',save = experiment+'_'+layer+'_ViolinPlot_Down_TB.pdf',ax=ax)

    fig,ax = plt.subplots(figsize=(5,3))
    ax.set_ylim(0,7)
    sc.pl.violin(sc_data,['pct_counts_NotUpNearTB'],groupby='leiden',rotation=90,inner='quartile',save = experiment+'_'+layer+'_ViolinPlot_NOTUP_TB.pdf',ax=ax)
             
    print(df_results.p_value[df_results.p_value>0].min())
    df_results.loc[df_results.p_value==0,'p_value'] = 0.9/perm_iterations


    df_results.to_csv('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Stats/stats_'+experiment+'_'+layer+'.csv')

    plt.scatter(x = df_results.sample_stat, y=-np.log10(df_results.p_value))
    plt.xlabel('difference in means, Bin1 - Null')
    plt.ylabel('-log p value')

    for i in range(df_results.shape[0]):
     plt.text(x=df_results.sample_stat[i],y=-np.log10(df_results.p_value[i]),s=df_results.index[i], 
          fontdict=dict(color='black',size=4))

    plt.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/volcano_'+experiment+'_'+layer+'_'+'.pdf')

    print(df_results.sort_values('sample_stat',ascending=True)[:20])
    print(df_results.sort_values('sample_stat',ascending=False)[:20])   


    marker_dict = {'High':['Nos2', 'Cd74', 'Irf1','C1qb'], 'Low':['Sparc','Jun','Lyve1','Ly6e'],'Neither':['Irf9','H2K1','Cxcr2']}


    #run for each set of marker genes
    for celltype in list(marker_dict.keys()):
   
        for transcript in marker_dict[celltype]:
    #if transcript not in ['Nos2','C1qa','C1qb','C1qc','Fcgr4','Lyve1']:
        #continue

        
  
            fig,ax = plt.subplots(figsize=(5,3))
            sns.histplot(data = adata_df,
                 x=transcript, 
                 bins=10000, 
                 binrange=(0,max(adata_df[transcript])),
                 stat="density",
                 element="step", 
                 fill=False, 
                 cumulative=True,
                color='Grey',
                alpha=0.5)
        
    
            sns.histplot(data = adata_df[adata_df.Bin!='Null'],
                 x=transcript, 
                 hue='Bin', 
                 bins=10000, 
                 binrange=(0,max(adata_df[transcript])),
                 stat="density",
                 element="step", 
                 fill=False, 
                 cumulative=True, 
                 common_norm=False,
                palette = ['Red','Orange','Green','Teal','Blue'])
            ax.set_xlabel(celltype)
            ax.set_ylabel('Cumulative Distribution')
            ax.set_title(transcript)
            ax.set_ylim(0,1)
            fig.savefig('/hpc/projects/data.science/leah.dorman/Spatial/Vizgen/Shoshana/Figures/'+experiment+'_'+transcript+'_'+layer+'_histogram.pdf')        
        
    print(experiment, "Up: ",sc_data.var.UpNearTB.value_counts())
    print("Down: ", sc_data.var.DownNearTB.value_counts())
    print("Neither: ",sc_data.var.NotUpNearTB.value_counts())


